In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')

## Load GGIR output 
(From script: `/Volumes/Untitled/rehab/GGIR/GGIR_LG-miar.Rmd`)

In [2]:
import pyreadr

In [3]:
diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-07 00:05:00'), pd.Timestamp('2024-03-07 06:36:00')], # 633 OK
    "906": [pd.Timestamp('2024-03-07 00:30:00'), pd.Timestamp('2024-03-07 07:30:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 06:00:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 23:15:00'), pd.Timestamp('2024-03-14 06:50:00')], # 127 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
    "547": [pd.Timestamp('2024-03-16 01:04:00'), pd.Timestamp('2024-03-16 07:40:00')], # 547 OK
    "815": [pd.Timestamp('2024-03-20 23:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK
    "914": [pd.Timestamp('2024-03-20 21:50:00'), pd.Timestamp('2024-03-21 05:50:00')], # 914 OK
    "971": [pd.Timestamp('2024-03-20 23:50:00'), pd.Timestamp('2024-03-21 07:50:00')], # 971 OK
    "279": [pd.Timestamp('2024-03-28 00:10:00'), pd.Timestamp('2024-03-28 07:27:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:25:00'), pd.Timestamp('2024-03-28 09:20:00')], # 965 OK
}

diary_TIB = {
    "158": [pd.Timestamp('2024-02-28 22:15:00'), pd.Timestamp('2024-02-29 07:45:00')], # 158 OK
    "633": [pd.Timestamp('2024-03-06 23:39:00'), pd.Timestamp('2024-03-07 08:00:00')], # 633 OK 
    "906": [pd.Timestamp('2024-03-07 00:15:00'), pd.Timestamp('2024-03-07 07:35:00')], # 906 OK
    "958": [pd.Timestamp('2024-03-13 21:30:00'), pd.Timestamp('2024-03-14 06:30:00')], # 958 OK
    "127": [pd.Timestamp('2024-03-13 22:00:00'), pd.Timestamp('2024-03-14 07:10:00')], # 127 OK 
    "098": [pd.Timestamp('2024-03-16 01:49:00'), pd.Timestamp('2024-03-16 09:52:00')], # 098 OK 
    "547": [pd.Timestamp('2024-03-16 00:26:00'), pd.Timestamp('2024-03-16 08:20:00')], # 547 OK 
    "815": [pd.Timestamp('2024-03-20 22:00:00'), pd.Timestamp('2024-03-21 07:30:00')], # 815 OK 
    "914": [pd.Timestamp('2024-03-20 21:30:00'), pd.Timestamp('2024-03-21 06:20:00')], # 914 OK 
    "971": [pd.Timestamp('2024-03-20 23:30:00'), pd.Timestamp('2024-03-21 08:08:00')], # 971 OK 
    "279": [pd.Timestamp('2024-03-28 00:04:00'), pd.Timestamp('2024-03-28 07:41:00')], # 279 OK
    "965": [pd.Timestamp('2024-03-28 01:22:00'), pd.Timestamp('2024-03-28 09:22:00')], # 965 OK
}

In [4]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
bursts_all_limbs = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}
for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    # end time - onset time
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]
    # onset time - previous end time
    # print(min(SIB_all[sub].dropna()["awake.duration"]))
    # SIB[sub]['sib.onset.time'] += pd.Timedelta('5s')
    # SIB[sub]['sib.end.time'] += pd.Timedelta('5s')
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_TIB.pkl', 'rb') as f:
        bursts = pickle.load(f)
    bursts_lw[sub] = bursts["lw"]
    bursts_rw[sub] = bursts["rw"]
    bursts_ll[sub] = bursts["ll"]
    bursts_rl[sub] = bursts["rl"]
    bursts_trunk[sub] = bursts["trunk"]
    bursts_all_limbs[sub] = bursts["all_limbs"]

    spt_start = diary_TIB[sub][0]
    spt_end = diary_TIB[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    # Angle-z
    metashort = pd.read_csv(part2_outputFolder + "/metashortLW_" + sub + ".CWA.RData.csv", index_col=1)
    metashort.index = pd.to_datetime(metashort.index).tz_localize(None)
    anglez = metashort.drop(columns=["Unnamed: 0", "ENMO"])

    SIB[sub]["sub_ID"] = sub
    bursts_lw[sub]["sub_ID"] = sub
    bursts_rw[sub]["sub_ID"] = sub
    bursts_ll[sub]["sub_ID"] = sub
    bursts_rl[sub]["sub_ID"] = sub
    bursts_trunk[sub]["sub_ID"] = sub
    bursts_all_limbs[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    bursts_lw[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_lw[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_lw[sub].at[j, "sib"] = 1
    bursts_rw[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_rw[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_rw[sub].at[j, "sib"] = 1
    bursts_ll[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_ll[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_ll[sub].at[j, "sib"] = 1
    bursts_rl[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_rl[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_rl[sub].at[j, "sib"] = 1
    bursts_trunk[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_trunk[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_trunk[sub].at[j, "sib"] = 1
    bursts_all_limbs[sub]["sib"] = 0
    for i, sib in SIB[sub].iterrows():
        for j, burst in bursts_all_limbs[sub].iterrows():
            if burst["start"] > sib["sib.onset.time"] + pd.Timedelta('5s') and burst["end"] < sib["sib.end.time"] - pd.Timedelta('5s'):
                bursts_all_limbs[sub].at[j, "sib"] = 1

## EDA - Single limbs but burst may happen together

SIB contains the SIB for all subjects (start time, end time and duration). Obviously, it contains also the AWAKE periods (total of n-1 events). 

In [9]:
# One DataFrame for all subjects
SIB_all = pd.concat(SIB.values(), ignore_index=True)
SIB_all["awake.duration.seconds"] = SIB_all["awake.duration"].dt.total_seconds()
SIB_all["sib.duration.seconds"] = SIB_all["sib.duration"].dt.total_seconds()
SIB_all["sib.duration.minutes"] = SIB_all["sib.duration"].dt.total_seconds() / 60
SIB_all["awake.duration.minutes"] = SIB_all["awake.duration"].dt.total_seconds() / 60

SIB and bursts overlap - 1 subject

In [8]:
plt.figure()
plt.plot(anglez)
for i, sib in SIB[sub].iterrows():
    plt.axvspan(sib["sib.onset.time"], sib["sib.end.time"], color='r', alpha=0.5)
for j, burst in bursts_lw[sub].iterrows():
    plt.axvspan(burst["start"], burst["end"], color='b', alpha=0.5)

Duration of awake periods between SIB

In [10]:
# Plot the number of awake periods between SIB
sns.set_context("talk")

plt.figure()
sns.stripplot(data=SIB_all, x="sub_ID", y="awake.duration.minutes", jitter=0.2, alpha = 0.5, color = "blue")
sns.boxplot(data=SIB_all, x="sub_ID", y="awake.duration.minutes", showfliers=False, color = "y")
plt.ylabel("Awake duration (minutes)")

plt.title("Awake periods between SIB during Time In Bed")

Text(0.5, 1.0, 'Awake periods between SIB during Time In Bed')

Duration of SIB

In [8]:
# Plot the number of SIB
sns.set_context("talk")

plt.figure()
sns.stripplot(data=SIB_all, x="sub_ID", y="sib.duration.minutes", jitter=0.2, alpha = 0.5, color = "blue")
sns.boxplot(data=SIB_all, x="sub_ID", y="sib.duration.minutes", showfliers=False, color = "pink")
plt.ylabel("SIB duration (minutes)")

plt.title("SIB during Time In Bed")

Text(0.5, 1.0, 'SIB during Time In Bed')

Get a burst DataFrame for all subjects for each limb

In [9]:
bursts_lw_allSub = pd.concat(bursts_lw.values(), ignore_index=True)
bursts_lw_allSub["duration_seconds"] = bursts_lw_allSub["duration"].dt.total_seconds()
bursts_rw_allSub = pd.concat(bursts_rw.values(), ignore_index=True)
bursts_rw_allSub["duration_seconds"] = bursts_rw_allSub["duration"].dt.total_seconds()
bursts_ll_allSub = pd.concat(bursts_ll.values(), ignore_index=True)
bursts_ll_allSub["duration_seconds"] = bursts_ll_allSub["duration"].dt.total_seconds()
bursts_rl_allSub = pd.concat(bursts_rl.values(), ignore_index=True)
bursts_rl_allSub["duration_seconds"] = bursts_rl_allSub["duration"].dt.total_seconds()
bursts_trunk_allSub = pd.concat(bursts_trunk.values(), ignore_index=True)
bursts_trunk_allSub["duration_seconds"] = bursts_trunk_allSub["duration"].dt.total_seconds()

Number of bursts happening during sleep (SIB) and during wake

In [10]:
burst_count_lw = bursts_lw_allSub["sib"].groupby(bursts_lw_allSub["sub_ID"]).value_counts().unstack()
burst_count_rw = bursts_rw_allSub["sib"].groupby(bursts_rw_allSub["sub_ID"]).value_counts().unstack()
burst_count_ll = bursts_ll_allSub["sib"].groupby(bursts_ll_allSub["sub_ID"]).value_counts().unstack()
burst_count_rl = bursts_rl_allSub["sib"].groupby(bursts_rl_allSub["sub_ID"]).value_counts().unstack()
burst_count_trunk = bursts_trunk_allSub["sib"].groupby(bursts_trunk_allSub["sub_ID"]).value_counts().unstack()

sns.set_context("talk")

# Bar plot
plt.figure(figsize = (15, 12))
plt.subplot(3, 2, 1)
burst_count_lw.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 2)
burst_count_rw.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 3)
burst_count_ll.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 4)
burst_count_rl.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)
plt.subplot(3, 2, 5)
burst_count_trunk.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)

from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 19, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of bursts")
plt.xticks(rotation = 19)
plt.suptitle("Number of bursts during SIB and Awake Periods")
plt.tight_layout()


Posture changes (at least 30°) happening during sleep and during wake

In [13]:
# Posture changes

bursts_lw_posture_changes = bursts_lw_allSub[bursts_lw_allSub["posture_change"] > 30]

bursts_posture_changes_counts = bursts_lw_posture_changes["sib"].groupby(bursts_lw_allSub["sub_ID"]).value_counts().unstack()

plt.figure()
bursts_posture_changes_counts.plot(kind='bar', color = ['blue', 'red'])
from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 19, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of posture changes")
plt.xticks(rotation = 19)
plt.title("Posture changes during SIB and Awake Periods")

Text(0.5, 1.0, 'Posture changes during SIB and Awake Periods')

Burst entity (AUC) during sleep and during wake 

In [11]:
import seaborn as sns
sns.set_context('talk')

# Set up the matplotlib figure
plt.figure(figsize=(19, 12))

# Boxplot for AUC
plt.subplot(3, 2, 1)
ax1 = sns.boxplot(x='sib', y='AUC', data=bursts_lw_allSub, hue = "sub_ID", showfliers=False)
plt.title('LW')
plt.ylabel('Movement Intensity (AUC)')
plt.xlabel(None)
ax1.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 2)
ax2 = sns.boxplot(x='sib', y='AUC', data=bursts_rw_allSub, hue = "sub_ID", showfliers=False)
plt.title('RW')
plt.ylabel(None)
plt.xlabel(None)
ax2.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 3)
ax3 = sns.boxplot(x='sib', y='AUC', data=bursts_ll_allSub, hue = "sub_ID", showfliers=False)
plt.title('LL')
plt.ylabel(None)
plt.xlabel(None)
ax3.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.subplot(3, 2, 4)
ax4 = sns.boxplot(x='sib', y='AUC', data=bursts_rl_allSub, hue = "sub_ID", showfliers=False)
plt.title('RL')
plt.ylabel(None)
plt.xlabel(None)
ax4.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])
handles, labels = [], []

for handle, label in zip(*ax4.get_legend_handles_labels()):
    if label not in labels:
        handles.append(handle)
        labels.append(label)

plt.legend(handles, labels, loc='lower right', frameon=True, fancybox=True, shadow=True, bbox_to_anchor=(0.8, -1.1), ncol=3, fontsize = 18)

plt.subplot(3, 2, 5)
ax5 = sns.boxplot(x='sib', y='AUC', data=bursts_trunk_allSub, hue = "sub_ID", showfliers=False)
plt.title('Trunk')
plt.ylabel(None)
plt.xlabel(None)
ax5.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

plt.show()

## EDA - Bursts happening in all limbs together

Merge the bursts of all subjects into one DataFrame

In [5]:
bursts_all_limbs_allSub = pd.concat(bursts_all_limbs.values(), ignore_index=True)

Number of bursts happening during sleep (SIB) and during wake

In [54]:
burst_count = bursts_all_limbs_allSub["sib"].groupby(bursts_all_limbs_allSub["sub_ID"]).value_counts().unstack()
sns.set_context("talk")

# Bar plot
plt.figure(figsize = (15, 12))
burst_count.plot(kind='bar', color = ['blue', 'red'], ax = plt.gca())
plt.legend([])
plt.ylabel("N of bursts")
plt.xticks(rotation = 19)

from matplotlib.patches import Patch

custom_patches = [Patch(facecolor="b"),
                Patch(facecolor="r")]

plt.legend(custom_patches, ["Awake", "Asleep"], loc = 'best', fontsize = 24, frameon = True, fancybox = True, shadow = True)
plt.ylabel("Number of bursts")
plt.xticks(rotation = 19)
plt.suptitle("Number of bursts during SIB and Awake Periods")
plt.tight_layout()

Burst entity (AUC) during sleep and during wake 

In [40]:
import seaborn as sns
sns.set_context('talk')

# Set up the matplotlib figure
plt.figure(figsize=(19, 12))

# Boxplot for AUC
ax1 = sns.boxplot(x='sib', y='AUC', data=bursts_all_limbs_allSub, hue = "sub_ID", showfliers=False)
plt.title('Movement Entity during sleep and wake')
plt.ylabel('Burst Entity (AUC)')
plt.xlabel(None)
ax1.legend_.remove()
plt.xticks([0, 1], ['Awake', 'Asleep'])

handles, labels = [], []

for handle, label in zip(*ax4.get_legend_handles_labels()):
    if label not in labels:
        handles.append(handle)
        labels.append(label)

plt.legend(handles, labels, loc='upper right', frameon=True, fancybox=True, shadow=True, ncol=3, fontsize = 21)

Percentiles of AUC

In [113]:
burst_describe = bursts_all_limbs_allSub.groupby(["sub_ID", "sib"])["AUC"].describe(percentiles = [0.9]).unstack()
burst_describe

count             mean                 std                 min  \
sib         0      1         0         1         0         1         0   
sub_ID                                                                   
098      44.0   20.0  6.653740  0.554661  7.152739  0.848130  0.087940   
158      63.0   44.0  5.404391  0.821867  8.144159  1.343849  0.072168   
279      18.0   19.0  7.570457  0.521787  9.849087  0.470343  0.196278   
547      58.0   27.0  6.906781  0.394636  7.425048  0.554090  0.066033   
633     138.0   67.0  3.676741  0.538104  4.664902  1.051418  0.033980   
815     143.0  110.0  4.272575  0.653986  9.421159  1.212396  0.028514   
906      39.0   46.0  3.971676  0.928337  5.091755  1.849077  0.049147   
958      53.0   32.0  4.028301  0.853181  3.692125  1.447341  0.189213   
971      47.0   30.0  3.183710  0.342831  3.814307  0.479453  0.050280   

                       50%                  90%                  max            
sib            1         0         1          0         1          0         1  
sub_ID                                                                          
098     0.036440  4.312882  0.219282  15.466636  1.699520  32.016583  3.147857  
158     0.020699  2.458432  0.337410  12.912712  2.124245  41.595520  7.972749  
279     0.022541  3.636635  0.410731  19.048691  1.270420  36.922621  1.393919  
547     0.016537  4.881188  0.143086  17.244877  1.287616  35.248337  1.977761  
633     0.016485  1.679356  0.142984  10.269015  1.647083  23.853609  5.364481  
815     0.022363  2.218856  0.207835   8.016730  1.829797  90.555946  8.442378  
906     0.020909  2.010235  0.154564   9.270132  2.312497  26.160641  8.204787  
958     0.024885  2.471979  0.153863   8.968217  2.740480  16.743444  6.185938  
971     0.024210  1.755934  0.160333   8.352138  0.633358  14.629342  2.113501

In [115]:
# Are there awake burts with less or equal entity than the maximum asleep burst?

AUC_sleep90 = burst_describe["90%"][1]

In [129]:
burst_awake_low_AUC = {sub: 0 for sub in subjects}
burst_sleep_low_AUC = {sub: 0 for sub in subjects}
for i, sub in enumerate(subjects):
    bursts_awake = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 0]
    bursts_sleep = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 1]

    # Find bursts with less or equal entity than the maximum asleep burst
    burst_awake_low_AUC[sub] = bursts_awake[bursts_awake["AUC"] <= AUC_sleep90[sub]]
    burst_sleep_low_AUC[sub] = bursts_sleep#bursts_sleep[bursts_sleep["AUC"] <= AUC_sleep90[sub]]

# One DataFrame for all subjects
bursts_awake_low_AUC_all = pd.concat(burst_awake_low_AUC.values(), ignore_index=True)
bursts_sleep_low_AUC_all = pd.concat(burst_sleep_low_AUC.values(), ignore_index=True)

In [130]:
# merge the two DataFrames
bursts_low_AUC_all = pd.concat([bursts_awake_low_AUC_all, bursts_sleep_low_AUC_all], ignore_index=True)
bursts_low_AUC_all["sib"] = bursts_low_AUC_all["sib"].replace({0: "Awake", 1: "Asleep"})
bursts_low_AUC_all

,start,end,limbs_involved,AUC,posture_change,sub_ID,sib
0,2024-02-28 22:19:07.308890104,2024-02-28 22:19:15.331500053,"{ll, rw, rl, lw, trunk}",0.249701,NaN,158,Awake
1,2024-02-28 22:19:43.688889980,2024-02-28 22:20:18.178889990,"{ll, rw, rl, lw, trunk}",0.588274,NaN,158,Awake
2,2024-02-28 22:20:39.318890095,2024-02-28 22:21:01.828890085,"{ll, rw, rl, lw, trunk}",0.429421,NaN,158,Awake
3,2024-02-28 22:36:40.808890104,2024-02-28 22:36:53.901499987,"{ll, rw, rl, lw, trunk}",0.211791,NaN,158,Awake
4,2024-02-28 22:36:57.844870090,2024-02-28 22:37:23.088890076,"{ll, rw, rl, lw, trunk}",1.566868,NaN,158,Awake
...,...,...,...,...,...,...,...
649,2024-03-21 07:17:09.782916069,2024-03-21 07:17:13.544266224,"{ll, rw, rl, lw, trunk}",0.068493,NaN,815,Asleep
650,2024-03-21 07:19:17.504266499,2024-03-21 07:19:23.362916231,"{ll, rw, rl, lw, trunk}",0.064362,NaN,815,Asleep
651,2024-03-21 07:19:45.114266157,2024-03-21 07:19:51.562916517,"{ll, rw, rl, lw, trunk}",0.187400,NaN,815,Asleep
652,2024-03-21 07:20:13.392916203,2024-03-21 07:20:20.444266319,"{ll, rw, rl, lw, trunk}",0.203871,NaN,815,Asleep


In [122]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='sub_ID', y='AUC', data=bursts_awake_low_AUC_all, showfliers=False)
sns.stripplot(x='sub_ID', y='AUC', data=bursts_awake_low_AUC_all, jitter=0.2, alpha = 0.5, color = "red")
plt.subplot(1, 2, 1)
sns.boxplot(x='sub_ID', y='AUC', data=bursts_sleep_low_AUC_all, showfliers=False,  whis=[5, 90])
sns.stripplot(x='sub_ID', y='AUC', data=bursts_sleep_low_AUC_all, jitter=0.2, alpha = 0.5, color = "red")


<Axes: xlabel='sub_ID', ylabel='AUC'>

In [135]:
plt.figure(figsize=(15, 6))
sns.boxplot(x='sub_ID', y='AUC', data=bursts_low_AUC_all, hue = "sib")
plt.ylabel("Movement intensity (AUC)")

Text(0, 0.5, 'Movement intensity (AUC)')

In [141]:
bursts_awake_low_AUC_all.value_counts("sub_ID").sort_index()

sub_ID
098    12
158    30
279     7
547    13
633    69
815    67
906    21
958    28
971    12
Name: count, dtype: int64

In [142]:
bursts_sleep_low_AUC_all.value_counts("sub_ID").sort_index()


sub_ID
098     20
158     44
279     19
547     27
633     67
815    110
906     46
958     32
971     30
Name: count, dtype: int64

## HR response during wakefullness

Is there a dose-response relationship? (based on the three percentiles)

In [5]:
bursts_all_limbs_allSub = pd.concat(bursts_all_limbs.values(), ignore_index=True)

In [6]:
import neurokit2 as nk

from functions.HR_response import detect_HR_change_from_RR, coherent_avg
from functions.bursts import filter_bursts

In [ ]:
offsets = {"158": 2, "098": 2, "633": 2, "279": 3, "906": 2, "547": 2, "971": 2, "958": 2, "815": 2}

hr_responses_high = {}
hr_responses_med = {}
hr_responses_low = {}

hr_responses_awake = {}
hr_responses_awake_high = {}
hr_responses_awake_med = {}
hr_responses_awake_low = {}

hr_responses_sleep = {}
hr_responses_sleep_high = {}
hr_responses_sleep_med = {}
hr_responses_sleep_low = {}

for i, sub in enumerate(subjects):

    print(sub)

    off = offsets[sub]

    ecg_df = pd.read_pickle(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/ecg.pkl')
    start_sleep, end_sleep = diary_SPT[sub]

    ecg_df = ecg_df.loc[start_sleep:end_sleep]
    ecg_filtered = nk.ecg_clean(ecg_df.values, sampling_rate=130)

    # Extract peaks
    _, results = nk.ecg_peaks(ecg_filtered, sampling_rate=130, method = 'neurokit')
    rpeaks = results["ECG_R_Peaks"]
    _, rpeaks_corrected = nk.signal_fixpeaks(rpeaks, sampling_rate=130, iterative=True, method="Kubios")

    t_rpeaks = ecg_df.index.to_series().values[rpeaks]
    t_rpeaks_corrected = ecg_df.index.to_series().values[rpeaks_corrected]
    rr = np.diff(t_rpeaks).astype('timedelta64[ns]').astype('float64') / 1000000000
    rr_corrected = np.diff(t_rpeaks_corrected).astype('timedelta64[ns]').astype('float64') / 1000000000
    hr_ecg = 60/rr
    hr_ecg_corrected = 60/rr_corrected
    hr_df = pd.Series(hr_ecg_corrected, index = t_rpeaks_corrected[1:]).resample("1 s").mean()#.rolling('10s', min_periods=1, center=True).mean()
    hr_df = hr_df.interpolate(method = 'cubic')
    hr_df_noncorrected = pd.Series(hr_ecg, index = t_rpeaks[1:]).resample("1 s").mean()
    hr_df_noncorrected = hr_df_noncorrected.interpolate(method = 'linear')

    artifacts_ecg = pd.read_csv(f'/Volumes/Untitled/rehab/data/{sub}/polar_processed/artifacts_ecg.csv')
    artifacts_ecg['Start'] = pd.to_datetime(artifacts_ecg['Start']).apply(lambda x: x.replace(tzinfo=None))
    artifacts_ecg['End'] = pd.to_datetime(artifacts_ecg['End']).apply(lambda x: x.replace(tzinfo=None))

    for i in range(len(artifacts_ecg)):
        hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    hr_df.interpolate(method = 'cubic', inplace = True)

    for i in range(len(artifacts_ecg)):
        if artifacts_ecg["End"].iloc[i] - artifacts_ecg["Start"].iloc[i] > pd.Timedelta("20 s"):
            hr_df.loc[artifacts_ecg["Start"].iloc[i]:artifacts_ecg["End"].iloc[i]] = np.nan

    #### PLOT #### 
    # plt.figure(figsize=(15, 9))
    # plt.plot(hr_df, label = "HR corrected")
    # for i, artifact in artifacts_ecg.iterrows():
    #     plt.axvspan(artifact["Start"], artifact["End"], color = "red", alpha = 0.3)
    # plt.title(f"HR corrected {sub}")

    # SIB_sub = SIB[sub]
    # for i, sib in SIB_sub.iterrows():
    #     plt.axvspan(sib["sib.onset.time"], sib["sib.end.time"], color = "green", alpha = 0.3)

    # plt.vlines(bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 0]["start"], ymin = 0, ymax = 200, color = "blue", alpha = 0.3)
    
    bursts_awake = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 0]
    bursts_sleep = bursts_all_limbs[sub][bursts_all_limbs[sub]["sib"] == 1]
    # Cut according to start_sleep and end_sleep
    bursts_all = bursts_all_limbs[sub][(bursts_all_limbs[sub]["start"] >= start_sleep) & (bursts_all_limbs[sub]["end"] <= end_sleep)]
    bursts_awake = bursts_awake[(bursts_awake["start"] >= start_sleep) & (bursts_awake["end"] <= end_sleep)]
    bursts_sleep = bursts_sleep[(bursts_sleep["start"] >= start_sleep) & (bursts_sleep["end"] <= end_sleep)]

    HR_bursts, _, AUC, _ = detect_HR_change_from_RR(filter_bursts(bursts_all), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_awake, _, AUC_awake, _ = detect_HR_change_from_RR(filter_bursts(bursts_awake), hr_df, offsets[sub]+1, plot = False)
    HR_bursts_sleep, _, AUC_sleep, _ = detect_HR_change_from_RR(filter_bursts(bursts_sleep), hr_df, offsets[sub]+1, plot = False)

    # Small vs medium vs large movements (regarless of sleep or wake)
    AUC_33 = np.percentile(AUC, 33)
    AUC_66 = np.percentile(AUC, 66)

    HR_bursts_low = [HR_bursts[i] for i in range(len(HR_bursts)) if AUC[i] <= AUC_33]
    HR_bursts_med = [HR_bursts[i] for i in range(len(HR_bursts)) if (AUC[i] > AUC_33) and (AUC[i] <= AUC_66)]
    HR_bursts_high = [HR_bursts[i] for i in range(len(HR_bursts)) if AUC[i] > AUC_66]

    HR_change, _ = coherent_avg(HR_bursts)
    HR_change_low, _ = coherent_avg(HR_bursts_low)
    HR_change_med, _ = coherent_avg(HR_bursts_med)
    HR_change_high, _ = coherent_avg(HR_bursts_high)

    # Small vs medium vs large movements during wake
    AUC_33_awake = np.percentile(AUC_awake, 33)
    AUC_66_awake = np.percentile(AUC_awake, 66)

    HR_bursts_awake_low = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if AUC_awake[i] <= AUC_33_awake]
    HR_bursts_awake_med = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if (AUC_awake[i] > AUC_33_awake) and (AUC_awake[i] <= AUC_66_awake)]
    HR_bursts_awake_high = [HR_bursts_awake[i] for i in range(len(HR_bursts_awake)) if AUC_awake[i] > AUC_66_awake]

    HR_change_awake, _ = coherent_avg(HR_bursts_awake)
    HR_change_awake_low, _ = coherent_avg(HR_bursts_awake_low)
    HR_change_awake_med, _ = coherent_avg(HR_bursts_awake_med)
    HR_change_awake_high, _ = coherent_avg(HR_bursts_awake_high)

    # Small vs medium vs large movements during sleep
    AUC_33_sleep = np.percentile(AUC_sleep, 33)
    AUC_66_sleep = np.percentile(AUC_sleep, 66)

    HR_bursts_sleep_low = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if AUC_sleep[i] <= AUC_33_sleep]
    HR_bursts_sleep_med = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if (AUC_sleep[i] > AUC_33_sleep) and (AUC_sleep[i] <= AUC_66_sleep)]
    HR_bursts_sleep_high = [HR_bursts_sleep[i] for i in range(len(HR_bursts_sleep)) if AUC_sleep[i] > AUC_66_sleep]

    HR_change_sleep, _ = coherent_avg(HR_bursts_sleep)
    HR_change_sleep_low, _ = coherent_avg(HR_bursts_sleep_low)
    HR_change_sleep_med, _ = coherent_avg(HR_bursts_sleep_med)
    HR_change_sleep_high, _ = coherent_avg(HR_bursts_sleep_high)

    # Save the results
    hr_responses_high[sub] = HR_change_high
    hr_responses_med[sub] = HR_change_med
    hr_responses_low[sub] = HR_change_low   
    hr_responses_awake[sub] = HR_change_awake
    hr_responses_awake_low[sub] = HR_change_awake_low
    hr_responses_awake_med[sub] = HR_change_awake_med
    hr_responses_awake_high[sub] = HR_change_awake_high
    hr_responses_sleep[sub] = HR_change_sleep
    hr_responses_sleep_low[sub] = HR_change_sleep_low
    hr_responses_sleep_med[sub] = HR_change_sleep_med
    hr_responses_sleep_high[sub] = HR_change_sleep_high

In [9]:
import matplotlib.ticker as mtick

## All responses

In [10]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys
high_hr_matrix = np.array([hr_responses_high[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])
med_hr_matrix = np.array([hr_responses_med[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])
low_hr_matrix = np.array([hr_responses_low[key] for key in keys if ~np.isnan(hr_responses_high[key]).all()])

time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_high_hr = np.mean(high_hr_matrix, axis=0)
mean_med_hr = np.mean(med_hr_matrix, axis=0)
mean_low_hr = np.mean(low_hr_matrix, axis=0)
sem_high_hr = np.std(high_hr_matrix, axis=0, ddof=1) / np.sqrt(high_hr_matrix.shape[0])
sem_med_hr = np.std(med_hr_matrix, axis=0, ddof=1) / np.sqrt(med_hr_matrix.shape[0])
sem_low_hr = np.std(low_hr_matrix, axis=0, ddof=1) / np.sqrt(low_hr_matrix.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots()
f.set_figheight(7)
f.set_figwidth(21)

ax1.errorbar(time_seconds, mean_low_hr, yerr=sem_low_hr, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_med_hr, yerr=sem_med_hr, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_high_hr, yerr=sem_high_hr, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_low_hr, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.plot(time_seconds, mean_med_hr, color = 'r', marker = 'o', linewidth=2, label = "Medium Movement")  # Medium
ax1.plot(time_seconds, mean_high_hr, color = 'b', marker = 'o', linewidth=2, label = "Large movement")     # High

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

## Sleep and wake responses

In [20]:
keys = list(hr_responses_awake.keys())  # Assuming all dictionaries have the same keys

# keys = list(hr_responses_med.keys())  # Assuming all dictionaries have the same keys
high_hr_matrix_awake = np.array([hr_responses_awake_high[key] for key in keys if ~np.isnan(hr_responses_awake_high[key]).all()])
med_hr_matrix_awake = np.array([hr_responses_awake_med[key] for key in keys if ~np.isnan(hr_responses_awake_high[key]).all()])
low_hr_matrix_awake = np.array([hr_responses_awake_low[key] for key in keys if ~np.isnan(hr_responses_awake_high[key]).all()])
high_hr_matrix_awake = np.array([hr_responses_sleep_high[key] for key in keys if ~np.isnan(hr_responses_sleep_high[key]).all()])
med_hr_matrix_awake = np.array([hr_responses_sleep_med[key] for key in keys if ~np.isnan(hr_responses_sleep_high[key]).all()])
low_hr_matrix_awake = np.array([hr_responses_sleep_low[key] for key in keys if ~np.isnan(hr_responses_sleep_high[key]).all()])
time_seconds = np.arange(-19,40)  # Time axis

# Calculate mean HR trends for each category
mean_high_hr_awake = np.mean(high_hr_matrix_awake, axis=0)
mean_med_hr_awake = np.mean(med_hr_matrix_awake, axis=0)
mean_low_hr_awake = np.mean(low_hr_matrix_awake, axis=0)
sem_high_hr_awake = np.std(high_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(high_hr_matrix_awake.shape[0])
sem_med_hr_awake = np.std(med_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(med_hr_matrix_awake.shape[0])
sem_low_hr_awake = np.std(low_hr_matrix_awake, axis=0, ddof=1) / np.sqrt(low_hr_matrix_awake.shape[0])

# Create boxplots for each HR type with transparency and customized outlier properties
f, (ax1) = plt.subplots()
f.set_figheight(7)
f.set_figwidth(21)

ax1.errorbar(time_seconds, mean_low_hr_awake, yerr=sem_low_hr_awake, fmt='g-', linewidth=2, label=None, ecolor='green', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_med_hr_awake, yerr=sem_med_hr_awake, fmt='r-', linewidth=2, label=None, ecolor='red', elinewidth=1.5, capsize=3)
ax1.errorbar(time_seconds, mean_high_hr_awake, yerr=sem_high_hr_awake, fmt='b-', linewidth=2, label=None, ecolor='blue', elinewidth=1.5, capsize=3)

# Add mean HR trend lines
ax1.plot(time_seconds, mean_low_hr_awake, color = 'g', marker = 'o', linewidth=2, label = "Small movement")  # Low
ax1.plot(time_seconds, mean_med_hr_awake, color = 'r', marker = 'o', linewidth=2, label = "Medium Movement")  # Medium
ax1.plot(time_seconds, mean_high_hr_awake, color = 'b', marker = 'o', linewidth=2, label = "Large movement")     # High

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelleft=True, labelsize=20)

ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")
plt.grid(True)
plt.tight_layout()

In [49]:
plt.figure()
# plt.plot(HR_change_awake, label = "All")
plt.plot(HR_change_awake_low, label = "Low")
plt.plot(HR_change_awake_med, label = "Medium")
plt.plot(HR_change_awake_high, label = "High")
plt.legend()

In [9]:
HR_response

(array([-1.01977427, -0.9034987 , -0.98065599, -0.39465717,  0.11418064,
        -0.21386798,  0.3119101 ,  0.7951134 ,  0.60426608, -0.03173758,
         0.6989472 ,  0.77625677, -0.03850405, -0.13950623,  0.98272396,
         1.07363257,  1.81395161,  2.80623378,  3.7392958 ,  3.85556033,
         5.5012593 ,  9.96850198, 14.01516982, 17.4354767 , 20.55308448,
        22.93782339, 21.39673735, 21.74220184, 20.81504811, 21.40692982,
        21.28389351, 21.23632315, 22.25316566, 23.45917556, 23.25017683,
        23.53122181, 22.31534356, 21.2536938 , 19.28035867, 16.91175056,
        14.62035761, 12.82396905,  9.60516659,  8.10273621,  8.33619697,
         6.96497898,  6.00360123,  5.69306217,  5.0414513 ,  4.63414262,
         3.01011967,  1.34293011,  1.61275477,  1.76751582,  0.37302696,
        -0.43043203, -0.73555399, -1.27467217, -0.75167606]),
 array([ 4.69942723,  5.63905666,  4.92058215,  5.71146143,  4.24108033,
         3.38180644,  3.16545493,  3.64258254,  4.2312007 ,  6

In [12]:
plt.figure()
plt.plot(HR_response[0])